In [ ]:
url = 'https://media1.vocaroo.com/mp3/1n5KKFJs9A8n'
duration_minutes = 1 # minutes

In [ ]:
!pip3 install --upgrade pip
!pip3 install webrtcvad librosa scipy audiosegment tqdm
!pip3 install --upgrade holoviews panel tqdm
!conda install -c conda-forge librosa -y

In [ ]:
import librosa
import soundfile as sf
from scipy.io import wavfile
from pathlib import Path
import webrtcvad
import io
from urllib.request import urlopen
import struct
import audiosegment
import numpy as np
import collections
import contextlib
import sys
import wave
import webrtcvad
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import numpy as np
from scipy.io import wavfile
from scipy.signal import spectrogram


def audiosegment_to_librosawav(audio_segment):
    channel_sounds = audio_segment.split_to_mono()
    return channels_to_librosawav(channel_sounds)


def channels_to_librosawav(channel_sounds):
    samples = [s.get_array_of_samples() for s in channel_sounds]

    fp_arr = np.array(samples).T.astype(np.float32)
    fp_arr /= np.iinfo(samples[0].typecode).max
    fp_arr = fp_arr.reshape(-1)

    return fp_arr

# https://github.com/wiseman/py-webrtcvad/blob/master/example.py
def read_wave(path):
    """Reads a .wav file.
    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate


def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)


class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.
    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.
    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.
    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.
    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.
    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.
    Arguments:
    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).
    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    if triggered:
        sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])

In [ ]:
start_ms = 0
duration_ms = int(duration_minutes * 60 * 1000) # ms
input_filename = "notebook_data/tmp.mp3"
resampled_filename = "notebook_data/tmp.wav"
resampled_filename_mp3 = "notebook_data/tmp.mp3" 
webm_filename = "notebook_data/tmp.webm"
Path(input_filename).parent.mkdir(parents=True, exist_ok=True)
Path(input_filename).write_bytes(urlopen(url).read())
audio = audiosegment.from_file(input_filename)[start_ms: start_ms + duration_ms]
audio.export(webm_filename, format="webm")
audio.export(resampled_filename_mp3, format="mp3")
sample_rate = audio.frame_rate
samples = audiosegment_to_librosawav(audio)

new_sample_rate = 16000
samples = librosa.resample(samples, sample_rate, new_sample_rate)
sample_rate = new_sample_rate
sf.write(resampled_filename, samples, sample_rate)
sample_rate, samples = wavfile.read(resampled_filename)
vad = webrtcvad.Vad()

# set aggressiveness from 0 to 3
mode = 3
vad.set_mode(mode)
raw_samples = struct.pack("%dh" % len(samples), *samples)
audio, sample_rate = read_wave(resampled_filename)

vad = webrtcvad.Vad(mode)
frames = frame_generator(30, audio, sample_rate)
frames = list(frames)
segments = vad_collector(sample_rate, 30, 300, vad, frames)
for i, segment in enumerate(segments):
    path = 'chunk-%002d.wav' % (i,)
    write_wave(path, segment, sample_rate)

window_duration = 0.03 # duration in seconds
samples_per_window = int(window_duration * sample_rate + 0.5)
bytes_per_sample = 2
segments = []

for start in np.arange(0, len(samples), samples_per_window):
    stop = min(start + samples_per_window, len(samples))
    
    is_speech = vad.is_speech(raw_samples[start * bytes_per_sample: stop * bytes_per_sample], 
                              sample_rate = sample_rate)

    segments.append(dict(
       start = start,
       stop = stop,
       is_speech = is_speech))
    
fig = plt.figure(figsize = (10,7))
ax = plt.gca()
ax.plot(samples)

ymax = max(samples)

# plot segment identifed as speech
x = np.array([])
y = np.array([])
label = np.zeros(len(samples), dtype=int)
recs = []

ax = fig.axes[0]
for segment in segments:
    if segment['is_speech']:
        label[segment['start'] : segment['stop'] - 1] = 1
        ax.add_patch(Rectangle((segment['start'], ymax), segment['stop'] - 1 - segment['start'] , 2000, edgecolor = 'orange',facecolor = 'orange',fill=True, zorder=3))
for r in recs:
    ax.add_patch(r)
plt.xlabel('sample')
plt.grid()
plt.savefig('tmp.png', dpi=150)

In [ ]:
def get_merged_speech_segments(segments, sample_rate):
    sr = sample_rate
    start = -1
    stop = -1
    segs = []
    speech_segments = [s for s in segments if s['is_speech']]
    for s in speech_segments:
        if stop == s['start']:
            stop = s['stop']
        else:
            if start >= 0 and stop >= 0:
                segs += [{'start': start / sr, 'stop': stop / sr}]
            start = s['start']
            stop = s['stop']
    if start >= 0 and stop >= 0:
                segs += [{'start': start / sr, 'stop': stop / sr}]
    return segs
merged = get_merged_speech_segments(segments, sample_rate)

In [ ]:
# Run it on Binder: https://mybinder.org/v2/gh/parkitny/notebook_demos/main

In [ ]:
import jp_proxy_widget
 
widget = jp_proxy_widget.JSProxyWidget()
 
js = "https://unpkg.com/wavesurfer.js"
js2="https://unpkg.com/wavesurfer.js/dist/plugin/wavesurfer.spectrogram.min.js"
js3="https://unpkg.com/wavesurfer.js/dist/plugin/wavesurfer.regions.min.js"
#url = "https://ia902606.us.archive.org/35/items/shortpoetry_047_librivox/song_cjrg_teasdale_64kb.mp3"

with open(input_filename, 'rb') as f:
    data = f.read()
    
widget.load_js_files([js, js2, js3])
 
widget.js_init("""
element.empty();
_regions = []
for(var i in intervals) {  
    console.log(intervals[i]['start'])
    console.log(intervals[i]['stop'])
    _regions.push({ 
            start: intervals[i]['start'],
            end: intervals[i]['stop'],
            loop: false,
            color: 'hsla(400, 100%, 30%, 0.5)',
            drag: false,
            resize: false
    });
}

function b64toBlob(b64Data, contentType, sliceSize) {
  contentType = contentType || 'image/png';
  sliceSize = sliceSize || 512;

  var byteCharacters = atob(b64Data);
  var byteArrays = [];

  for (var offset = 0; offset < byteCharacters.length; offset += sliceSize) {
    var slice = byteCharacters.slice(offset, offset + sliceSize);
  
    var byteNumbers = new Array(slice.length);
    for (var i = 0; i < slice.length; i++) {
        byteNumbers[i] = slice.charCodeAt(i);
    }
  
    var byteArray = new Uint8Array(byteNumbers);
  
    byteArrays.push(byteArray);
  }

  var blob = new Blob(byteArrays, {type: contentType});
  return blob;
}

function str2ab(str) {
  var buf = new ArrayBuffer(str.length*2); // 2 bytes for each char
  var bufView = new Uint16Array(buf);
  for (var i=0, strLen=str.length; i < strLen; i++) {
    bufView[i] = str.charCodeAt(i);
  }
  return buf;
}

var _blob = b64toBlob(data_str, '')

element.wavesurfer = WaveSurfer.create({
    container: element[0],
    waveColor: 'violet',
        progressColor: 'purple',
        loaderColor: 'purple',
        cursorColor: 'navy',
        minPxPerSec: 100,
        scrollParent: true,
        plugins: [
        WaveSurfer.spectrogram.create({
            wavesurfer: element.wavesurfer,
            container: element[0],
            fftSamples:512,
            labels: true
        }),
        WaveSurfer.regions.create({
            regions: _regions,
            dragSelection: {
                slop: 5
            }
        })
    ]
}); 

element.wavesurfer.loadBlob(_blob);
element.wavesurfer.on('ready', function () {
    element.wavesurfer.play();
});
""", url=url, data_str=data, intervals=merged)
 
widget